In [1]:
import os

In [2]:
%pwd

'd:\\LaboratoireDeveloppement_TI\\IA\\Machine Learning\\deeplearningproject\\Chicken-Disease-Classification-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\LaboratoireDeveloppement_TI\\IA\\Machine Learning\\deeplearningproject\\Chicken-Disease-Classification-Project'

In [5]:
# pip install keras

In [6]:
import os
import sys
import time
import tensorflow as tf
from pathlib import Path
from src.constants import *
from zipfile import ZipFile
import urllib.request as request
from dataclasses import dataclass
from src.logging.logger import logging
from src.exception.exception import CustomException
from src.utils.common import fn_read_yaml, fn_create_directories


from tensorflow.keras import backend
backend.clear_session()

# Ensure eager execution is enabled
tf.config.run_functions_eagerly(True)


@dataclass(frozen=True)
class _TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    
@dataclass(frozen=True)
class _PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path  
    
# tf.executing_eagerly()

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = fn_read_yaml(config_filepath)
        self.params = fn_read_yaml(params_filepath)      
        
        fn_create_directories([self.config.artifacts_root])
        
    def fn_get_prepare_callback_config(self) -> _PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        
        fn_create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])
        
        prepare_callback_config = _PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
            
        return prepare_callback_config
    
    def fn_get_training_config(self) -> _TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        
        fn_create_directories([Path(training.root_dir)])
        
        training_config = _TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        
        return training_config

In [8]:
class PrepareCallback:
    def __init__(self, config: _PrepareCallbacksConfig):
        self.config = config
        
        
    @property
    def fn_create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    
    @property
    def fn_create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
        
    def fn_get_tb_ckpt_callbacks(self):
        return [
            self.fn_create_tb_callbacks,
            self.fn_create_ckpt_callbacks
        ]

In [9]:
class Training:
    def __init__(self, config: _TrainingConfig):
        self.config = config       
        
    def fn_get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )    
        
    def fn_train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                zoom_range = 0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
            
        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        # model.converters.save_keras_model(path)
        
    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list,
            verbose=1
        )
        
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:    
    # tf.compat.v1.enable_eager_execution()
    # tf.config.run_functions_eagerly(True)
    logging.info("Initializing configuration...")
    config = ConfigurationManager()

    logging.info("Preparing callbacks...")
    prepare_callbacks_config = config.fn_get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.fn_get_tb_ckpt_callbacks()

    logging.info("Loading training configuration...")
    training_config = config.fn_get_training_config()
    training = Training(config=training_config)

    logging.info("Loading base model...")
    training.fn_get_base_model()

    logging.info("Preparing data generators...")
    training.fn_train_valid_generator()

    logging.info("Starting training...")
    training.train(callback_list=callback_list)

    logging.info("Training complete. Model saved.")
    
except Exception as e:
    raise CustomException(e, sys)

Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


c:\Users\User\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\User\anaconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


CustomException: Error occurred in python script name [C:\Users\User\AppData\Local\Temp\ipykernel_22396\2860701499.py] line number [23] error message [Unknown variable: <Variable path=dense/kernel, shape=(25088, 2), dtype=float32, value=[[ 0.01454542  0.00262116]
 [ 0.01113208  0.01499637]
 [ 0.0083723  -0.01148277]
 ...
 [-0.01432328  0.01014562]
 [-0.00523406 -0.0031569 ]
 [ 0.01142745 -0.00160965]]>. This optimizer can only be called for the variables it was originally built with. When working with a new set of variables, you should recreate a new optimizer instance.]